In [1]:
# 导入赛题所需库文件
from sklearn.model_selection import train_test_split
import shutil, random, json, os, cv2
from collections import Counter
import xml.dom.minidom as xmldom
import pandas as pd


In [2]:
# 读取标记数据
data = pd.read_csv('./data/2/图片虫子位置详情表.csv', encoding='GBK')
data.head()

,序号,文件名,虫子编号,虫子名称,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标
0,1,00001.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00004.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
2,3,00006.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
3,4,00008.jpg,7,二化螟,1984.0,2150.0,1835.0,1920.0,2134.0,2380.0
4,5,00008.jpg,228,二点委夜蛾,3301.0,827.0,3040.0,618.0,3563.0,1037.0


In [3]:
# 利用字典实现虫子编号转换
bh = ['6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148', '156', '222',
      '228', '235', '256', '280', '310', '387', '392', '394', '398', '401', '402', '430', '480', '485', '673', '0']
bh2 = [str(i) for i in range(28)] + ['111']
dd = {bh[i]: bh2[i] for i in range(29)}

data['虫子编号'] = data['虫子编号'].apply(lambda x: dd[str(x)])
data.head()

,序号,文件名,虫子编号,虫子名称,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标
0,1,00001.jpg,111,无,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00004.jpg,111,无,NaN,NaN,NaN,NaN,NaN,NaN
2,3,00006.jpg,111,无,NaN,NaN,NaN,NaN,NaN,NaN
3,4,00008.jpg,1,二化螟,1984.0,2150.0,1835.0,1920.0,2134.0,2380.0
4,5,00008.jpg,13,二点委夜蛾,3301.0,827.0,3040.0,618.0,3563.0,1037.0


In [8]:
# 坐标标准化
data['x'] = 5472
data['y'] = 3648
data['中心点x坐标'] = data['中心点x坐标']/data['x']
data['中心点y坐标'] = data['中心点y坐标']/data['y']
data['左上角x坐标'] = data['左上角x坐标']/data['x']
data['左上角y坐标'] = data['左上角y坐标']/data['y']
data['右下角x坐标'] = data['右下角x坐标']/data['x']
data['右下角y坐标'] = data['右下角y坐标']/data['y']

data['中心点x坐标'] = data['中心点x坐标'].apply(lambda x: 0.9999 if x > 1 else x)
data['中心点y坐标'] = data['中心点y坐标'].apply(lambda x: 0.9999 if x > 1 else x)
data['左上角x坐标'] = data['左上角x坐标'].apply(lambda x: 0.9999 if x > 1 else x)
data['左上角y坐标'] = data['左上角y坐标'].apply(lambda x: 0.9999 if x > 1 else x)
data['右下角x坐标'] = data['右下角x坐标'].apply(lambda x: 0.9999 if x > 1 else x)
data['右下角y坐标'] = data['右下角y坐标'].apply(lambda x: 0.9999 if x > 1 else x)

# 获取长度、宽度信息
data['xxx'] = data['右下角x坐标'] - data['左上角x坐标']
data['yyy'] = data['右下角y坐标'] - data['左上角y坐标']

data.head()

,序号,文件名,虫子编号,虫子名称,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标,x,y,xxx,yyy
0,1,00001.jpg,111,无,NaN,NaN,NaN,NaN,NaN,NaN,5472,3648,NaN,NaN
1,2,00004.jpg,111,无,NaN,NaN,NaN,NaN,NaN,NaN,5472,3648,NaN,NaN
2,3,00006.jpg,111,无,NaN,NaN,NaN,NaN,NaN,NaN,5472,3648,NaN,NaN
3,4,00008.jpg,1,二化螟,0.362573,0.589364,0.335344,0.526316,0.389985,0.652412,5472,3648,0.054642,0.126096
4,5,00008.jpg,13,二点委夜蛾,0.603253,0.226700,0.555556,0.169408,0.651133,0.284265,5472,3648,0.095577,0.114857


In [9]:
# 保存为Yolo V3输入格式 //后面发现预测效果太差没有使用V3
data = data.astype(str)
d = {}
for i in range(data.shape[0]):
    if data['虫子名称'][i] != '无':
        d[data['文件名'][i]] = d.get(data['文件名'][i],[]) + [','.join(data[['虫子编号','中心点x坐标','中心点y坐标','xxx','yyy']].iloc[i].tolist())]
    else:
        d[data['文件名'][i]] = []

with open('./data/label.txt','w') as f:
    for key in d.keys():
        f.write(key + ' ' + ' '.join(d[key]) + '\n')

FileNotFoundError: [Errno 2] No such file or directory: './dataset/label.txt'

In [ ]:
# 文件路径
path = ',/images/'
# 训练所用图片文件名保存
names = data['文件名'].tolist()
# 训练集与验证集划分
train, val = train_test_split(names,train_size=0.8)
with open('train.txt','w') as f:
    for i in train:
        f.write(path + i + '\n')
with open('val.txt','w') as f:
    for i in val:
        f.write(path + i + '\n')


In [7]:
# YOLO V5格式标签保存
d = {}     # 利用字典存储文件名与标签对应关系
data = data.astype(str)   # float转str方便后续操作
for i in range(data.shape[0]):
    if data['虫子名称'][i] != '无':
        d[data['文件名'][i]] = d.get(data['文件名'][i],[]) + [' '.join(data[['虫子编号','中心点x坐标','中心点y坐标','xxx','yyy']].iloc[i].tolist())]
    else:
        d[data['文件名'][i]] = []

# 从字典获取文件名
k = list(d.keys())
# 将标签依次保存为YOLO V5训练所需txt文件格式
for name in k:
    with open('./data/labels/' + name.split('/')[-1].replace('jpg','txt'),'w') as f:
        for i in d[name]:
            f.write(i + '\n')

,序号,文件名,虫子编号,虫子名称,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标,x,y,xxx,yyy
0,1,00001.jpg,111,无,nan,nan,nan,nan,nan,nan,5472,3648,nan,nan
1,2,00004.jpg,111,无,nan,nan,nan,nan,nan,nan,5472,3648,nan,nan
2,3,00006.jpg,111,无,nan,nan,nan,nan,nan,nan,5472,3648,nan,nan
3,4,00008.jpg,1,二化螟,0.36257309941520466,0.3929093567251462,0.33534356725146197,0.5263157894736842,0.38998538011695905,0.6524122807017544,5472,3648,0.054641812865497075,0.1260964912280702
4,5,00008.jpg,13,二点委夜蛾,0.6032529239766082,0.1511330409356725,0.5555555555555556,0.16940789473684212,0.6511330409356725,0.28426535087719296,5472,3648,0.0955774853801169,0.11485745614035084


In [ ]:
# 保存除零以外的虫子编号
name = list(d)
# 转换为int类型方便排序
name = [int(i) for i in name]
# 去除零标签
name.remove(0)
# 种类排序
name.sort()
# 标签保存
with open('./data/classes.txt','w') as f:
    for i in name:
        f.write(str(i)+'\n')

In [ ]:
name = [str(i) for i in name]
print(name)

In [61]:


test = pd.read_csv('./data/3/无位置信息的图片汇总表.csv',encoding='GBK')
path = './img_test/'

test = list(test['文件名'].apply(lambda x:path + x))
with open('test.txt','w') as f:
    for i in val:
        f.write(path + i + '\n')

In [ ]:
for name in test:
    # 复制测试图片至指定文件夹
    shutil.copy(name,'./data/img_test/' + name.split('/')[-1])

In [ ]:
with open('./data/test.txt','w') as f:
    for i in test:
        f.write(str(i)+'\n')

In [11]:
with open('./data/test.txt','w') as f:
    for i in test:
        f.write(str(i)+'\n')

In [21]:


test = pd.read_csv('./data/3/无位置信息的图片汇总表.csv',encoding='GBK')
path = '../data/test/'

test = list(test['文件名'].apply(lambda x:path + x))

for name in test:
    # 复制测试图片至指定文件夹
    shutil.copy(name,'./data/img_test/' + name.split('/')[-1])

In [ ]:
with open('./data/test.txt','w') as f:
    for i in test:
        f.write(str(i)+'\n')

In [ ]:
with open('./data/test.txt','w') as f:
    for i in test:
        f.write(str(i)+'\n')

In [50]:
name = [str(i) for i in name]
print(name)

['6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148', '156', '222', '228', '235', '256', '280', '310', '387', '392', '394', '398', '401', '402', '430', '480', '485', '673']


In [94]:
# 保存无位置信息的图片文件名，作为测试集
test = pd.read_csv('./data/3/无位置信息的图片汇总表.csv', encoding='GBK')
path = './img_test/'
# 文件名前加上文件路径
test = list(test['文件名'].apply(lambda x: path + x))
# 测试集保存
with open('./data/test.txt', 'w') as f:
    for i in test:
        f.write(str(i) + '\n')

In [ ]:
# 将自制标签xml格式的文件转化为txt文件格式

# 类型
classes = ['6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148', '156',
           '222', '228', '235', '256', '280', '310', '387', '392', '394', '398',
           '401', '402', '430', '480', '485', '673']


def data2txt(d1, d2, d3, d4, clsName, txtFile_name):
    fd = open(txtFile_name, 'a')
    index = classes.index(clsName)
    fd.write(str(index) + " " + str(d1) + " " + str(d2) + " " + str(d3) + " " + str(d4) + "\n")
    fd.close()


# 读取xml文件数据
def xml2data(xmlfile_name, txtFile_name):
    print(xmlfile_name)
    ele = xmldom.parse(os.path.abspath(xmlfile_name)).documentElement

    size = ele.getElementsByTagName("size")
    if len(size) != 0:
        image_width = int(size[0].getElementsByTagName("width")[0].firstChild.data)
        image_height = int(size[0].getElementsByTagName("height")[0].firstChild.data)

    objList = ele.getElementsByTagName("object")
    for obj in objList:
        index = objList.index(obj)
        clsName = objList[index].getElementsByTagName("name")[0].firstChild.data
        bndbox = objList[index].getElementsByTagName("bndbox")
        xmin = int(bndbox[0].getElementsByTagName("xmin")[0].firstChild.data)
        ymin = int(bndbox[0].getElementsByTagName("ymin")[0].firstChild.data)
        xmax = int(bndbox[0].getElementsByTagName("xmax")[0].firstChild.data)
        ymax = int(bndbox[0].getElementsByTagName("ymax")[0].firstChild.data)

        xcenter = (xmax + xmin) / 2
        ycenter = (ymax + ymin) / 2
        width = abs(xmax - xmin)
        height = abs(ymax - ymin)

        data2txt(xcenter / image_width, ycenter / image_height, width / image_width, height / image_height, clsName,
                 txtFile_name)


def readFile(filePath):
    xmlDir = filePath + "/" + 'xml'
    if (False == os.path.exists(xmlDir)):
        return None
    txtDir = filePath + "/" + 'txt'
    if (False == os.path.exists(txtDir)):
        os.mkdir(txtDir)
    for fileName in os.listdir(xmlDir):
        xml2data(xmlDir + '/' + fileName, txtDir + '/' + fileName[:-4] + ".txt")


readFile("./data/label/")

In [20]:
# 保存自制数据集文件及路径信息
path = './data/my_images/'
images = os.listdir(path)
with open('./data/my_images.txt', 'w') as f:
    for img in images:
        f.write('./my_images/' + img + '\n')

10-0.txt
10-1.txt
10-10.txt
10-11.txt
10-12.txt
10-13.txt
10-14.txt
10-15.txt
10-16.txt
10-17.txt
10-18.txt
10-19.txt
10-2.txt
10-3.txt
10-4.txt
10-5.txt
10-6.txt
10-8.txt
10-9.txt
105-0.txt
105-1.txt
105-10.txt
105-11.txt
105-12.txt
105-13.txt
105-14.txt
105-15.txt
105-16.txt
105-17.txt
105-18.txt
105-19.txt
105-2.txt
105-20.txt
105-21.txt
105-22.txt
105-23.txt
105-24.txt
105-25.txt
105-26.txt
105-27.txt
105-3.txt
105-4.txt
105-5.txt
105-6.txt
105-7.txt
105-8.txt
105-9.txt
110-0.txt
110-10.txt
110-100.txt
110-101.txt
110-102.txt
110-103.txt
110-104.txt
110-106.txt
110-107.txt
110-108.txt
110-109.txt
110-11.txt
110-110.txt
110-112.txt
110-113.txt
110-114.txt
110-115.txt
110-116.txt
110-12.txt
110-120.txt
110-122.txt
110-123.txt
110-125.txt
110-126.txt
110-127.txt
110-128.txt
110-13.txt
110-131.txt
110-132.txt
110-133.txt
110-134.txt
110-135.txt
110-136.txt
110-137.txt
110-139.txt
110-14.txt
110-140.txt
110-142.txt
110-143.txt
110-144.txt
110-145.txt
110-146.txt
110-147.txt
110-15.txt
1

In [9]:
# 检测结果json文件解析
with open('best_predictions.json', 'r') as f:
    data = json.load(f)
image, category, bbox, score = [], [], [], []
for i in data:
    if i['score'] > 0.3:
        image.append(i['image_id'])
        category.append(i['category_id'])
        bbox.append(i['bbox'])
        score.append(i['score'])

d = {}
names = [ '6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148',
         '156', '222', '228', '235', '256', '280', '310', '387', '392',
         '394', '398', '401', '402', '430', '480', '485', '673' ]
for i in range(28):
    d[i] = names[i]
data = pd.DataFrame({'image':image, 'category':category, 'bbbox':bbox, 'score':score})
data.head()

In [16]:
# 最终训练集及验证集划分
train,val,test = [],[],[]
classes = [ '6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148',
         '156', '222', '228', '235', '256', '280', '310', '387', '392',
         '394', '398', '401', '402', '430', '480', '485', '673' ]
# 0.8,0.2

for cls in classes:
    data_new = data[data['虫子编号'] == int(cls)]
    names = data_new['文件名'].tolist()
    if len(names) > 1:
        train1, val1 = train_test_split(names, train_size=0.8)
        train.extend(train1)
        val.extend(val1)
    else:
        test.extend(names)

flag = 0
for name in train:
    if name in val and flag == 0:
        train.remove(name)
        flag = 1 - flag
    if name in val and flag == 1:
        val.remove(name)
        flag = 1 - flag

data_new = data[data['虫子编号'] == 0]
names = data_new['文件名'].tolist()
if len(names) > 1:
    train1, val1 = train_test_split(names, train_size=0.8)
    train.extend(train1)
    val.extend(val1)

path = './images/'
train = [path + name for name in train]
val = [path + name for name in val]

with open('train.txt', 'w') as f:
    for name in train:
        f.write(name + '\n')
with open('val.txt', 'w') as f:
    for name in val:
        f.write(name + '\n')

## 结果保存为问题二所需格式

In [21]:
path = './data/final/'
names = os.listdir(path)
print(names)
x, y = 5472, 3648

classes = ['6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148', '156',
           '222', '228', '235', '256', '280', '310', '387', '392', '394', '398',
           '401', '402', '430', '480', '485', '673']


cls, dirs, x_zs, y_zs, x_mins, y_mins, x_maxs, y_maxs, pres = [], [], [], [], [], [], [], [], []
for name in names:
    bbox = []
    with open(path + name, 'r') as f:
        lines = f.readlines()
    for line in lines:
        cl, x_z, y_z, width, length, pre = map(eval, line.strip().split(' '))

        x_min = x * (x_z - width/2)
        x_max = x * (x_z + width/2)
        y_min = y * (y_z - length/2)
        y_max = y * (y_z + length/2)

        x_z *= x
        y_z *= y

        dirs.append(name.replace('txt','jpg'))
        cls.append(classes[cl])
        x_mins.append(round(x_min))
        x_maxs.append(round(x_max))
        y_mins.append(round(y_min))
        y_maxs.append(round(y_max))
        x_zs.append(round(x_z))
        y_zs.append(round(y_z))
        pres.append(pre)


data = pd.DataFrame({'文件名':dirs,'虫子编号':cls,  '中心点x坐标':x_zs, '中心点y坐标':y_zs, '左上角x坐标':x_mins,
                     '左上角y坐标':y_mins, '右下角x坐标':x_maxs, '右下角y坐标':y_maxs, 'pre':pres})
data.head()

['00013.txt', '00042.txt', '00061.txt', '00070.txt', '00088.txt', '00103.txt', '00109.txt', '00142.txt', '00166.txt', '00190.txt', '00194.txt', '00198.txt', '00204.txt', '00209.txt', '00213.txt', '00220.txt', '00225.txt', '00255.txt', '00267.txt', '00275.txt', '00281.txt', '00287.txt', '00289.txt', '00310.txt', '00324.txt', '00349.txt', '00355.txt', '00358.txt', '00413.txt', '00435.txt', '00444.txt', '00450.txt', '00455.txt', '00497.txt', '00550.txt', '00556.txt', '00579.txt', '00588.txt', '00605.txt', '00623.txt', '00643.txt', '00680.txt', '00707.txt', '00723.txt', '00778.txt', '00799.txt', '00849.txt', '00863.txt', '00874.txt', '00931.txt', '00937.txt', '00950.txt', '00968.txt', '00977.txt', '00984.txt', '01044.txt', '01048.txt', '01057.txt', '01067.txt', '01117.txt', '01128.txt', '01133.txt', '01135.txt', '01151.txt', '01152.txt', '01184.txt', '01187.txt', '01199.txt', '01220.txt', '01232.txt', '01246.txt', '01259.txt', '01262.txt', '01287.txt', '01296.txt', '01325.txt', '01327.txt'

,文件名,虫子编号,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标,pre
0,00013.jpg,398,2987,807,2852,592,3121,1023,0.278623
1,00013.jpg,156,3019,2755,2571,2397,3467,3113,0.832369
2,00013.jpg,156,2373,380,1943,0,2802,759,0.862603
3,00042.jpg,9,1737,610,1684,577,1790,642,0.830077
4,00061.jpg,156,2669,1305,2263,1059,3076,1551,0.514470


In [22]:
data2 = pd.read_csv('./result.csv',encoding='GBK')
da = pd.merge(data2, data,how="outer")
da['虫子编号'] = da['虫子编号'].fillna(0)

da.head()

,序号,文件名,虫子编号,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标,pre
0,1,00012.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00013.jpg,398,2987.0,807.0,2852.0,592.0,3121.0,1023.0,0.278623
2,2,00013.jpg,156,3019.0,2755.0,2571.0,2397.0,3467.0,3113.0,0.832369
3,2,00013.jpg,156,2373.0,380.0,1943.0,0.0,2802.0,759.0,0.862603
4,3,00016.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:

da['序号'] = [i+1 for i in range(len(da))]
da.head(15)


,序号,文件名,虫子编号,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标,pre
0,1,00012.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00013.jpg,398,2987.0,807.0,2852.0,592.0,3121.0,1023.0,0.278623
2,3,00013.jpg,156,3019.0,2755.0,2571.0,2397.0,3467.0,3113.0,0.832369
3,4,00013.jpg,156,2373.0,380.0,1943.0,0.0,2802.0,759.0,0.862603
4,5,00016.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,00031.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,00041.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,00042.jpg,9,1737.0,610.0,1684.0,577.0,1790.0,642.0,0.830077
8,9,00047.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,00050.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
da.to_csv('result2.csv', encoding='GB2312', index=None)

## 结果保存为问题三所需格式

In [15]:
data_new = data[['文件名','虫子编号']]

xxx = []
for i in range(len(data)):
    xxx.append(data.iloc[i]['文件名'] + '-' + str(data.iloc[i]['虫子编号']))
d = Counter(xxx)
print(d)

Counter({'01816.jpg-156': 6, '01044.jpg-156': 3, '01152.jpg-156': 3, '01325.jpg-156': 3, '00013.jpg-156': 2, '00061.jpg-156': 2, '00070.jpg-156': 2, '00142.jpg-280': 2, '00588.jpg-156': 2, '00605.jpg-7': 2, '00623.jpg-7': 2, '00968.jpg-10': 2, '01117.jpg-7': 2, '01199.jpg-7': 2, '01232.jpg-156': 2, '01287.jpg-156': 2, '01325.jpg-7': 2, '01710.jpg-156': 2, '02265.jpg-156': 2, '02570.jpg-156': 2, '02702.jpg-156': 2, '02749.jpg-156': 2, '02872.jpg-156': 2, '03215.jpg-7': 2, '03440.jpg-7': 2, '03561.jpg-156': 2, '00013.jpg-398': 1, '00042.jpg-9': 1, '00061.jpg-7': 1, '00088.jpg-10': 1, '00088.jpg-115': 1, '00103.jpg-7': 1, '00109.jpg-156': 1, '00166.jpg-480': 1, '00166.jpg-280': 1, '00190.jpg-256': 1, '00194.jpg-156': 1, '00198.jpg-10': 1, '00204.jpg-8': 1, '00204.jpg-228': 1, '00209.jpg-156': 1, '00213.jpg-7': 1, '00220.jpg-9': 1, '00220.jpg-10': 1, '00225.jpg-280': 1, '00225.jpg-7': 1, '00255.jpg-156': 1, '00267.jpg-10': 1, '00275.jpg-115': 1, '00281.jpg-256': 1, '00287.jpg-148': 1, '002

In [20]:
xxx = list(d.items())

names, cls, nums = [], [], []
for i in range(len(xxx)):
    names.append(xxx[i][0].split('-')[0])
    cls.append(xxx[i][0].split('-')[1])
    nums.append(xxx[i][1])
data_new = pd.DataFrame({'文件名': names, '虫子编号': cls, '数量': nums})
data_new.head(10)


data2 = pd.read_csv('./result.csv',encoding='GB2312')
da = pd.merge(data2, data_new,how="outer")
da['虫子编号'] = da['虫子编号'].fillna(0)
da['数量'] = da['数量'].fillna(0)
da['数量'] = da['数量'].astype(int)
da['序号'] = [i+1 for i in range(len(da))]
da.head(15)

da.to_csv('result3.csv', encoding='GBK', index=None)
da.head(15)

,序号,文件名,虫子编号,数量
0,1,00012.jpg,0,0
1,2,00013.jpg,398,1
2,3,00013.jpg,156,2
3,4,00016.jpg,0,0
4,5,00031.jpg,0,0
5,6,00041.jpg,0,0
6,7,00042.jpg,9,1
7,8,00047.jpg,0,0
8,9,00050.jpg,0,0
9,10,00061.jpg,156,2


In [10]:
file = os.listdir('D:/BaiduNetdiskDownload/测试数据/test')
id = [i for i in range(1, len(file) + 1)]

data = pd.DataFrame({'序号': id, '文件名': file})
data.to_csv('result.csv', index=None, encoding='GBK')
data

,序号,文件名
0,1,00012.jpg
1,2,00013.jpg
2,3,00016.jpg
3,4,00031.jpg
4,5,00041.jpg
...,...,...
795,796,03787.jpg
796,797,03790.jpg
797,798,03801.jpg
798,799,03805.jpg
